In [80]:
import pandas as pd

In [81]:
df = pd.read_csv("Loan_Train.csv")

In [82]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [83]:
df.shape

(614, 13)

##### Prepare the data for modeling by performing the following steps:

In [84]:
###### Drop the column “Load_ID.”

df = df.drop("Loan_ID",axis=1)

In [85]:
df.shape

(614, 12)

In [86]:
##### Drop any rows with missing data.
df = df.dropna()

In [87]:
df.shape

(480, 12)

In [88]:
### Convert the categorical features into dummy variables.
df.select_dtypes(include=['object']).columns.tolist()

['Gender',
 'Married',
 'Dependents',
 'Education',
 'Self_Employed',
 'Property_Area',
 'Loan_Status']

In [89]:
newdf = pd.get_dummies(df, columns=df.select_dtypes(include=['object']).columns.tolist(),drop_first=True)

In [90]:
newdf.shape

(480, 15)

In [91]:
newdf = newdf.rename(columns={'Loan_Status_Y':'Loan_Status'})

##### 3.Split the data into a training and test set, where the “Loan_Status” column is the target.

In [92]:
x_data = newdf.drop(['Loan_Status'],axis=1)
y_data = newdf['Loan_Status']

In [93]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data ,test_size = 0.2, shuffle=False)

##### 4.Create a pipeline with a min-max scaler and a KNN classifier (see section 15.3 in the Machine Learning with Python Cookbook).

In [94]:
from sklearn.preprocessing import MinMaxScaler

In [95]:
from sklearn.pipeline import Pipeline,FeatureUnion

In [133]:
# define min max scaler
scaler = MinMaxScaler()

In [134]:
from sklearn.neighbors import KNeighborsClassifier

In [187]:
knn = KNeighborsClassifier(n_neighbors=5,n_jobs=-1)

In [195]:
pipeline = Pipeline([("standardizer",scaler),("classifier",KNeighborsClassifier())])

##### 5.Fit a default KNN classifier to the data with this pipeline. 

In [196]:
pipeline.fit(x_train,y_train)

Pipeline(steps=[('standardizer', MinMaxScaler()),
                ('classifier', KNeighborsClassifier())])

In [197]:
pipeline.score(x_test,y_test)

0.75

In [191]:
##Model accuracy for the test set is 75%

##### 6.Create a search space for your KNN classifier where your “n_neighbors” parameter varies from 1 to 10. 

In [198]:
search_space = [{"classifier__n_neighbors": [1,2,3,4,5,6,7,8,9,10]}]

##### 7.Fit a grid search with your pipeline, search space, 
##### and 5-fold cross-validation to find the best value for the “n_neighbors” parameter.

In [193]:
from sklearn.model_selection import GridSearchCV

In [199]:
classifier = GridSearchCV(pipeline,search_space,cv=5,verbose=0).fit(x_train,y_train)

In [200]:
classifier.best_estimator_.get_params()["classifier__n_neighbors"]

4

##### 8.Find the accuracy of the grid search best model on the test set.

In [201]:
from sklearn.model_selection import cross_val_score
all_accuracies = cross_val_score(estimator=classifier, X=x_test, y=y_test, cv=5)

In [202]:
print(all_accuracies)

[0.75       0.78947368 0.78947368 0.78947368 0.68421053]


In [74]:
##Accuracy has improved

##### 9.Now, repeat steps 6 and 7 with the same pipeline, but expand your search space to include logistic regression and random forest models with the hyperparameter values

In [203]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [211]:
import numpy as np

In [225]:
search_space2 = [{"classifier":[KNeighborsClassifier()],"classifier__n_neighbors": [1,2,3,4,5,6,7,8,9,10]},
                 {"classifier": [LogisticRegression()],
                  "classifier_penalty":['l1','l2'],
                  "classifier_C":np.logspace(0,4,10)},
                 {"classifier":[RandomForestClassifier()],
                  "classifier_n_estimators":[10,100,1000],
                  "classifier_max_features":[1,2,3]}]

In [233]:
clf = GridSearchCV(pipeline, search_space2, cv = 5, verbose = 3,n_jobs=-1)

In [138]:
rf_clf_new = RandomForestClassifier(n_estimators=500)

# Fit this to the data and obtain predictions
rf_new_predictions = rf_clf_new.fit(x_train, y_train).predict(x_test)

In [144]:
from sklearn.linear_model import LogisticRegression
def lr_clf(C):
    model = LogisticRegression(C=C, random_state=0)
    scores = cross_val_score(model, x_train, y_train, cv=5)
    return scores.mean()
print(lr_clf(C=0.1))

0.7840054682159946


In [221]:
for param in clf.get_params().keys():
    print(param)

cv
error_score
estimator__memory
estimator__steps
estimator__verbose
estimator__standardizer
estimator__classifier
estimator__standardizer__clip
estimator__standardizer__copy
estimator__standardizer__feature_range
estimator__classifier__algorithm
estimator__classifier__leaf_size
estimator__classifier__metric
estimator__classifier__metric_params
estimator__classifier__n_jobs
estimator__classifier__n_neighbors
estimator__classifier__p
estimator__classifier__weights
estimator
n_jobs
param_grid
pre_dispatch
refit
return_train_score
scoring
verbose
